# CM4AI Community Detection Tutorial

In graph theory, terms like "communities," "clusters," "groups," and "modules" are often used interchangeably to describe subsets of nodes that are densely connected internally and sparsely connected to the rest of the network. This concept is central to community detection, which aims to identify such structures within graphs. Detecting these communities is crucial for understanding the organization and function of complex networks, as they often correspond to functional units within the system. 

Various algorithms have been developed to detect communities in networks. For instance, the Girvan–Newman algorithm identifies edges that lie between communities and removes them to reveal the community structure. Another popular method is modularity maximization, which searches for divisions of a network that have a high modularity score, indicating a strong community structure.

In this tutorial we will walk through how to perform community detection in a simple graph with three communities.

Along the way, we will learn about different metrics that are commonly used in Network Analysis which are related to this task. 

[Run In Google Colab](https://colab.research.google.com/github/tjdurant/cm4ai-quantum-comm-detect/blob/main/src/comm-detect-tutorial.ipynb)

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import community as community_louvain

## A Basic Graph with Three Communities

Let's start by drawing a simple graph and visually/subjectively deciding how many 'communities' we think there are. And for the sake of simplicity, lets make the decision to analyze this graph in a way where each node is assigned to a single, distinct community (non-ovelapping community detection).

In [2]:
def create_toy_graph():
    G = nx.Graph()
    G.add_edges_from([[1,2], [2,3], [1,3], [1,4], [1,5], [3,5],
                  [6,7], [7,8], [8,9], [6,9], [6,8], [7,9],
                  [10,11], [11,12], [10,12],
                  [9,12], [2,7]])
    return G

In [ ]:
G = create_toy_graph()
nx.draw(G, with_labels=True, node_color='r')

Most might argue there are three communities here (although some might argue that 4 is it's own community). 

Let's say there are <u>three</u> 'communities' in this graph.

So, how do we arrive at 'three communities' using a mathematical/algorithmic process?

Let's go through one method called the <u>Girvan-Newman Algorithm</u>. 

The first step is to calculate some metrics that help characterize the edges in this network. 

This is often done by using the general concept of centrality, which aims to quantify the “importance” of vertices or edges within a graph.

<br>
<br>
<br>


## <u>Edge Betweenness Centrality (EBC)</u> 

EBC asks the question, for every pair of nodes in the network, how many of the shortest paths between those nodes pass through a given edge. Stated a second way, it is a measure of the importance of an edge in a graph, calculated as the number of shortest paths between pairs of nodes that pass through that edge. It helps identify critical connections in networks.

Intuitively, an edge with high betweenness is "important" because it frequently appears in shortest paths connecting different vertices in the graph.

So, if we calculate for all edges, which would be the most important do you think?

In [ ]:
edge_centrality_G = nx.edge_betweenness_centrality(G)

sorted_by_values = dict(sorted(edge_centrality_G.items(), key=lambda x: x[1], reverse=True))

for edge, centrality in sorted_by_values.items():
    print(f"{edge}: {centrality}")

So, [2,7] is the most 'important'

Let's manually calculate edge betweenness for [2,7].
<br>
<br>
<br>
<br>

#### <u> Manually Calculate Edge Betweenness</u>

Remember, EBC is a measure of how many shortest paths (between all pairs of vertices) go through a particular edge. 

$$ b(e) = \sum_{s < t} \frac{\sigma_{s,t}(e)}{\sigma_{s,t}} $$

Where:

- $\sigma_{s,t}(e)$ is the number of those shortest paths that pass through the edge $e$.
- $\sigma_{s,t}$ denotes the total number of shortest paths between vertices $s$ and $t$.
- The summation $\sum_{s < t}$ is taken over all pairs of distinct vertices $s$ and $t$. 

For undirected graphs, it is common to sum over $s < t$ (instead of $s \neq t$) to avoid double-counting paths. The $s < t$ loops over every pair of distinct nodes in the graph exactly once rather than summing over both $(s,t)$ and $(t,s)$. This is purely a notational convenience for an undirected graph and ensures that every node pair is included **exactly once**, avoiding double-counting.

<br>
<br>
<br>

So what would $\sigma_{8,1}({2,7})$ be in the graph below?

In [ ]:
G3 = nx.Graph()
G3.add_edges_from([
    (1, 2), (1, 3),   # Connect 1 -> {2, 3}
    (2, 7), (3, 7),   # Connect {2,3} -> 7
    (7, 5), (7, 6), (7, 9),  # Connect 7 -> {5,6,9}
    (5, 8), (6, 8), (9, 8)   # Connect {5,6,9} -> 8
])
nx.draw(G3, with_labels=True, node_color='r')

all shortest paths from 1 to 8 have length 4, and exactly 6 such paths exist.

Hence, $\sigma_{1,8} = 6$

When we then consider the number of 'shortest paths' from 1 to 8 that pass through the edge [2,7], this would be $\sigma_{1,8}({2,7}) = 3$, giving us a 3/6 fraction for the (1,8) node pair.

Then we do this for all pairs of nodes, and sum all of these fractions with the $s < t$ constraint. 

<br>
<br>

In the originally proposed graph we can use some shortcuts.

Calculate the total number of possible node pairs in this network. Given that there are 12 nodes and we're looking for pairs (i.e., 2), we can use the '12 choose 2' calculation, simplified here:

  $$ \frac{12!}{2!(12-2)!} = 66 $$


In [ ]:
total_pairs = (12*11)/2
total_pairs

In [ ]:
# Then find the shortest paths for every pair and determine
# for every 'shortest path' which go through [2,7].
# For this graph we can calculate this via a shortcut by multiplying 
# by the number of nodes on either side of the [2,7] edge.
number_of_nodes_passing_through = 5*7
number_of_nodes_passing_through
 

So, 35 pairs of nodes in this network, rely on [2,7] on their shortest path to each other. 

In [ ]:
# If we divide these two numbers we get the edge betweenness
print(f"Edge Betweenness Centrality for edge [2, 7] in G by nx: {round(edge_centrality_G.get((2,7)),4)}")
print(f"Edge Betweenness Centrality for edge [2, 7] in G by manual: {round((number_of_nodes_passing_through / total_pairs),4)}")


So, [2,7] is the most 'important' as measured by this metric. 

But as a thought exercise, how could we decrease the centrality of that edge?

In [ ]:
# Add a path for 'the dangler' over to node 2 that would make it's shortest path not use the [2,7] edge
G2 = nx.Graph()
G2.add_edges_from([[1,2], [2,3], [1,3], [1,4], [1,5], [3,5],
                  [6,7], [7,8], [8,9], [6,9], [6,8], [7,9],
                  [10,11], [11,12], [10,12],
                  [9,12], [2,7], [4,7]])
nx.draw(G2, with_labels=True, node_color='r')

Here, we have added an edge between node 4 (i.e., 'the dangler') and node 7. In this network, the shortest path from 4 to 7 is no longer through the [2,7] edge. So the [2,7] calculation for edge betweenness becomes this:

In [ ]:
edge_centrality_G2 = nx.edge_betweenness_centrality(G2)
print(f"Edge Betweenness Centrality for edge [2, 7] in G: {round(edge_centrality_G.get((2,7)),4)}")
print(f"Edge Betweenness Centrality for edge [2, 7] in G2: {round(edge_centrality_G2.get((2,7)),4)}")

## <u>Girvan-Newman Algorithm</u>

Now that we have the EBC for all of the edges, how does the Girvan-Newman Algorithm use that to find communities?

Look for the edge that has the highest EBC and delete that edge. 

Recompute the EBC, and repeat. 

This is liberating communities from the network, so that hopefully at the end we have all of the communities by themselves. 

The trick is to know how many 'deletions' to do. 

In [ ]:
num_iterations = 2

G = create_toy_graph()

for i in range(num_iterations):
    tmp_ebc = nx.edge_betweenness_centrality(G).items()
    edge_to_delete = sorted(tmp_ebc, key=lambda pair: -pair[1])[0][0]
    
    G.remove_edge(*edge_to_delete)
    
    nx.draw(G, with_labels=True, node_color='r')
    plt.title('Step %s\nEdge %s Deleted'%(i, edge_to_delete), fontsize=20)
    
    plt.show()

In [ ]:
# --- Girvan-Newman Community Detection ---
# girvan_newman returns an iterator of partitions (each partition is a tuple of sets of nodes).
# The first `next(...)` will give the first split (2 communities). 
# You can keep calling `next(...)` to get more finely split communities.
G = create_toy_graph()

gn_partitions = nx.community.girvan_newman(G)
first_level_partition = next(gn_partitions)   # This is the partition with the first split
num_communities_gn = len(first_level_partition)
print("Number of communities (Girvan-Newman, first split):", num_communities_gn)

As you can see, the crux of using Girvan-Newman is deciding when to stop splitting up the graph. 

You typically track a quality measure (often modularity) at each step and pick the partition that yields the best score (e.g., the highest modularity). In other words, you have to decide the “optimal cut” or watch when the modularity peaks.

However, it can be computationally expensive for large networks, and you need to determine the best stopping point manually (by checking the modularity changes after each removal, or following some heuristic).

There are some algorithms where the decision of **when to stop** is baked into the algorithm and in a way that optimizes performance. 

But, before we do that, we need to understand another metric commonly used in Network Analysis: **<u>Modularity</u>**

<br>
<br>

---

## Modularity (Q)

### Intuitive Understanding of Modularity

Measures how well a graph is partitioned into communities. 

$Q \propto \sum_{s \in S} \Big[ (\text{number of edges within group } s) \;-\; (\text{number of EXPECTED edges within group } s) \Big]$

Intuitively, $Q$ is proporational to the summation over all the communities where for every community, we ask how many edges are there between the members of the group $s$, minus how many edges would I expect between the nodes of group $s$ in some random null model. 

And if the group $s$ connections are more than what we would expect in a random, then we have found a significant cluster.

The total modularity of the network is the sum of the modularity score for each individual cluster/community.

So, we need to figure a way to calculate the number of 'expected connections' within a community.
<br>
<br>

### Configuration Model

The Configuration Model is a random graph model that preserves the degree distribution of the original graph. 

It generates a network by randomly connecting nodes while ensuring that each node's degree matches its specified value, thereby maintaining the overall <u>degree distribution</u> of the network. This model provides a more realistic baseline for expected edges in networks with heterogeneous degree distributions.

The expected number of edges between two nodes $i $ and $j$ in the Configuration Model is given by:

$\text{Expected edges between } i \text{ and } j = \frac{k_i k_j}{2m}$

where:
- $k_i$ and $k_j$ are the degrees of nodes $i$ and $j$,
- $2m$ is the total number of edges in the network.

<br>
<br>


### Modularity Equation Breakdown

$Q(G, S) = \frac{1}{2m} \sum_{s \in S} \sum_{i \in s} \sum_{j \in s} \left( A_{ij} - \frac{k_i k_j}{2m} \right)$

Modularity a set of groups $S$ in graph $G$ is a sum over all the paris of nodes in the group, where we look at $A_{ij}$ to determine if that pair of nodes is connected in the group and then for every pair we calculate the expected number of edges between a pair of nodes using the Configuration Model as our null model. So we are saying what is the real number of edges, minus the expected number of edges, for all the pairs of nodes within a given group $s$, and now i sum this over all the groups $s$ into $S$. And $A_{ij}$ is 1 but if connection is 0. 

$\frac{1}{2m}$ is a normalizing constant that keeps $Q$ in the range of -1 to 1.
- Where if the modularity $Q$ for a given set of partitions $S$ is "high" (i.e., >0.5), the graph has significant community structure

<br>
<br>

### Why is Higher Modularity "Better"?

Modularity measures how well nodes within a single community connect to each other relative to how they connect to nodes outside that community. A higher modularity for a given community means:

- More internal cohesion: Most edges are between the nodes inside the community.
- Less external linkage: There are fewer edges connecting those nodes to other parts of the network.

If one finds that the summation of modularities is *"high"* across a set of partitions $S$ across a given graph $G$, the algorithm has likely done a good job at identifiying self-contained, distinct, or meaningful clusters within the larger network. 

The overall idea being that we have done a good job of identifing communities if we maximize modularity

So then this would become the target of an objective function that we use formulate as a kind of optimization problem for community detection - i.e., we will try to maximize $Q(G,S)$, and to do this we will use the <u>Louvain Algorithm</u>

<br>
<br>

-----

## The Louvain Algorithm

The **Louvain algorithm** is a popular method for detecting communities with high modularity scores $Q$ in large networks due to its **speed** and **scalability**. 

- *Greedy* algorithm for community detection
  - Refers to a strategy where decisions are made step-by-step, choosing the locally optimal solution at each step in the hope that this leads to a globally optimal solution.
- Supports weighted graphs
- Supports hierarchical clustering

<br>

It operates in two major phases that repeat iteratively:

1. **Local Modularity Optimization**  
   - Start with each node $i$ in its own community.  
   - For each node $i$, consider moving it to the community of one of its neighbors.  
   - Calculate the change in modularity $\Delta Q$ for this move. If $\Delta Q$ is **positive**, move node $i$ to that neighbor’s community.

2. **Community Aggregation**  
   - Once no more improvements can be made by local moves, **aggregate** each community into a “super-node.”  
   - Replace each entire community with a single super-node, and build a smaller, coarser network.  
   - Repeat Phase 1 on this aggregated network until no further modularity improvement is possible.

<br>
<br>

In [ ]:
G = create_toy_graph()

# --- Louvain Community Detection ---
# best_partition returns a dict of node -> community ID
louvain_partition = community_louvain.best_partition(G, resolution=1)
num_communities_louvain = len(set(louvain_partition.values()))
print("Number of communities (Louvain):", num_communities_louvain)

---

### Modularity and the Gain

In Louvain, the **modularity** $Q$ of a partition is commonly defined as:

$$
Q = \frac{1}{2m} \sum_{i,j} \biggl(A_{ij} - \frac{k_i \, k_j}{2m}\biggr) \delta(c_i, c_j),
$$

where:

- $A_{ij}$ is the adjacency matrix (1 if there is an edge between $i$ and $j$, or the weight if weighted).  
- $k_i$ is the degree (or sum of edge weights) of node $i$.  
- $m$ is the total number of edges (or total edge weight).  
- $\delta(c_i, c_j)$ is 1 if $i$ and $j$ are in the same community, and 0 otherwise.

When **moving** a single node $i$ from its current community to a neighboring community $C$, the **change in modularity** $\Delta Q$ can be computed (in one simplified form) as:

$$
\Delta Q \approx \frac{k_i^{\text{in},C} - k_i \frac{S_C}{2m}}{2m},
$$

where:

- $k_i^{\text{in},C}$ is the sum of the weights of edges from node $i$ to the nodes in community $C$.  
- $k_i$ is the sum of the weights of edges from node $i$ to **all** other nodes.  
- $S_C$ is the sum of the degrees (edge weights) of the nodes in community $C$.  
- $m$ is the total edge weight in the network.

If $\Delta Q > 0$, moving $i$ to $C$ **increases** overall modularity.

<br>
<br>


----

## Graph Partitioning Using Quantum Annealing

Following the methods described in [Negre et al.](https://www.osti.gov/pages/biblio/1628969), this process begins with creating an...

### <u>Adjacency Matrix</u>

$$
A_{ij} =
\begin{cases} 
0, & \text{if } i = j \\ 
w_{ij}, & \text{if } i \neq j 
\end{cases}
$$

1. $A_{ij}$ represents the element in the $i$-th row and $j$-th column of the matrix.
2. **Case 1:** If $i = j$, $A_{ij} = 0$. This means there are no self-loops in the graph, as the weight of an edge from a node to itself is zero.
3. **Case 2:** If $i \neq j$, $A_{ij} = w_{ij}$. Here, $w_{ij}$ is the weight of the edge connecting nodes $i$ and $j$. If no edge exists between $i$ and $j$, $w_{ij}$ is typically assumed to be zero.

Because our toy graph is unweighted, if $i \neq j$, $A_{ij} = 1$

In [ ]:
G = create_toy_graph()

A = nx.adjacency_matrix(G)
print ('\nAdjacency matrix:\n', A.todense())


Now we calculate... 

### <u>Node Degrees</u>

This begins with calculating the number of **degrees** per node. A degree is 'how many edges are touching a node'. 

Given by:

$k_i = \sum_{j} A_{ij}$

Where:
- $k_i$: Degree of node $i$
- $A_{ij}$: Element of the adjacency matrix representing the edge between nodes $i$ and $j$. So, here, we are summing the number of edges - which in an unweighted graph, each edge is equal to 1 - between $i$ and everyother node in the graph $j$. 

Below, you can see that node 0 is connected to 4 other nodes. 

In [ ]:
node_degrees = A.sum(axis=1)

print(f"{A.todense()}\n")
print(node_degrees.T)  

You can see above walk through some node examples:

$k_0 = 4$

$k_1 = 3$

And so on...

Now we calculate the node degrees over the whole graph, for every node. 

This is defined as:

$2m = \sum_i k_i = \sum_{ij} A_{ij}$

1. $k_i$: The degree of node $i$, which is the sum of weights of edges connected to $i$.
2. $\sum_i k_i$: The sum of all node degrees, which is twice the total edge weight for undirected graphs.
4. $\sum_{ij} A_{ij}$: An equivalent way of summing over all edges in the adjacency matrix $A$, counting each edge's weight twice for undirected graphs.

So, we can just use `.sum()` on `node_degrees` ($k_i$) to get node degree across whole graph ($2m$)

In [ ]:
two_m = node_degrees.sum()
two_m

<br>

Note that $m$ (little) is the total number of edges (or the total edge weight in the case of weighted graphs) in the graph

The total number of *node degrees* is $2m$

In an undirected graph, each edge is shared between two nodes. When summing the degrees ($k_i$) of all nodes, each edge is counted twice, once for each endpoint.

Mathematically this is stated as:

$$\sum_i k_i = 2m$$

This is sometimes called the <u>**handshaking lemma**</u> in graph theory.



Again, the total number of edges ($m$) is half of this number

In [ ]:
mtotal = two_m / 2.0
print("Sanity Check")
print(f"Total number of edges calculated from node_degrees (2m):    {int(mtotal)}")
print(f"Total number of edges calculated on the graph by nx:        {len(G.edges)} ")

Now we build the...

### <u>**Modularity Matrix**</u>

This is done by subtracting the adjacency matrix ($A_{ij}$) from the null model ($\frac{k_i k_j}{2m}$).

$
B_{ij} = A_{ij} - \frac{k_i k_j}{2m}
$

Lets calculate the *null model* first with generating a $k_i$ and $k_j$ matrix. 

The outer product of two vectors $\mathbf{k}$ and $\mathbf{k}$ is defined as:

$
\text{Outer product: } (\mathbf{k} \otimes \mathbf{k})_{ij} = k_i \cdot k_j
$

In Python, this is computed using `np.outer()`.

In [ ]:
# Note that k_j is:
node_degrees.T

In [ ]:
# So, to get the node_degree_matrix that is k_i * k_j, we can use np.outer():
np.outer(node_degrees, node_degrees)

Now we put it together:

In [ ]:
modularity_matrix = A - (np.outer(node_degrees, node_degrees) / two_m)

print ("\nModularity matrix: \n", modularity_matrix)

print ("min value = ", modularity_matrix.min())
print ("max value = ", modularity_matrix.max())

It is known that modularity-based optimization of graph partitioning can fail to detect communities that are small ([Fortunato 2007](https://www.pnas.org/doi/abs/10.1073/pnas.0605965104))

To help with this, we can add a 'resolution' parameter to our modularity matrix...

In [ ]:
resolution = 3
modularity_matrix = A - (resolution * np.outer(node_degrees, node_degrees) / two_m)

print ("\nModularity matrix: \n", modularity_matrix)

print ("min value = ", modularity_matrix.min())
print ("max value = ", modularity_matrix.max())

Higher resolution ($resolution > 1$): Amplifies the importance of modularity differences, encouraging smaller, finer-grained communities.

Lower resolution ($resolution < 1$):Reduces the weight of modularity differences, favoring larger communities.

#### Resolution in the Louvain Algorithm

In the Louvain algorithm:

- The **resolution parameter** modifies the modularity optimization process to control the size of the detected communities.

- A higher resolution parameter decreases the weight of the term $\frac{k_i \cdot k_j}{2m}$ (expected edges between nodes based on their degrees) relative to the actual edge weights. This encourages splitting large communities into smaller ones.

- The modified modularity function becomes:

$$
Q = \sum_{i,j} \left[ w_{ij} - \gamma \frac{k_i \cdot k_j}{2m} \right] \delta(c_i, c_j)
$$

where:

- $w_{ij}$: edge weight between nodes $i$ and $j$,  
- $k_i$: degree of node $i$,  
- $2m$: sum of all edge weights in the graph,  
- $\gamma$: resolution parameter (default is 1),  
- $\delta(c_i, c_j)$: Kronecker delta, 1 if $i$ and $j$ are in the same community, 0 otherwise.


<br>

------

So now that we have our Modularity Matrix ($B_{ij}$) we have to create the QUBO, or a...

### Quadratic Unconstrained Binary Optimization formulation

A QUBO is a mathematical formulation used in optimization problems. It expresses a problem as minimizing or maximizing a quadratic polynomial where the variables are binary (0 or 1). This formulation is often used in quantum computing, particularly for quantum annealers like D-Wave, because they solve problems expressed in this format.

if you don't have ground truth, and modularity is suboptimal in small community space, how do you measure performance or how do you know you are detecting the right communtiies. 

Which ones consume the most QPU time

How do you evaluate through a graph where you DON"T know the number of partitions beforehand - could just leave n-parts = n-nodes

could test hierarchical clustering with louvain 

Were there some hyperparameters taking up more time?

Why Is Louvain Faster Than Girvan–Newman?

1. **Local, Greedy Optimizations**  
   - Louvain updates communities by checking small, local moves for each node $i$.  
   - Girvan–Newman iteratively **removes the highest-betweenness edge** and recalculates global betweenness each time, which is computationally expensive.

2. **Hierarchical (Multi-Level) Approach**  
   - After no more local moves can improve modularity, Louvain **collapses** each community into a super-node and repeats the process on a smaller graph.  
   - This reduction significantly speeds up subsequent iterations.

3. **No Repeated Global Betweenness Computation**  
   - Girvan–Newman requires recomputing **edge betweenness** multiple times, which is costly ($O(n \times m)$ or worse).  
   - Louvain only needs to compute **local gains** in modularity ($\Delta Q$) when moving nodes.

4. **Early Termination**  
   - Louvain naturally **stops** once modularity cannot be improved any further.  
   - No need for exhaustive searches or repeated cuts of the graph.

As a result, **Louvain’s greedy, multi-level collapsing** approach scales well to large networks, making it **significantly faster** in practice than Girvan–Newman for community detection.